In [ ]:
import keras
import numpy as np 
import pandas as pd 
import zipfile
import matplotlib.pyplot as plt
import os
import shutil

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip", 'r') as zip_ref:
                zip_ref.extractall("./")

with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", 'r') as zip_ref:
                zip_ref.extractall("./")

In [ ]:
print("Total train images")
print(len([f for f in os.listdir('./train') if os.path.isfile(os.path.join('./train', f))]))
print("Total test images")
print(len([f for f in os.listdir('./test1') if os.path.isfile(os.path.join('./test1', f))]))

print("Total train cats images")
print(len([f for f in os.listdir('./train') if os.path.isfile(os.path.join('./train', f)) and 'cat' in f]))
print("Total train dogs images")
print(len([f for f in os.listdir('./train') if os.path.isfile(os.path.join('./train', f)) and 'dog' in f]))

print("Total test cats images")
print(len([f for f in os.listdir('./test1') if os.path.isfile(os.path.join('./test1', f)) and 'cat' in f]))
print("Total test dogs images")
print(len([f for f in os.listdir('./test1') if os.path.isfile(os.path.join('./test1', f)) and 'dog' in f]))

In [ ]:
i = 0
for dirname, _, filenames in os.walk('./train'):
    for filename in filenames:
        print(os.path.join(filename))
        img = plt.imread(os.path.join(dirname, filename))
        print(img.shape)
        plt.figure(i)
        plt.imshow(img)
        i+=1
        if i==7:
            break

In [ ]:
base_dir = './'

train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

train_cats = os.path.join(train_dir, 'cats')
os.mkdir(train_cats) 
train_dogs = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs)

In [ ]:
files = ['cat.{}.jpg'.format(i) for i in range(12500)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(train_cats, filename)
    shutil.copyfile(src, dst)
    
files = ['dog.{}.jpg'.format(i) for i in range(12500)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(train_dogs, filename)
    shutil.copyfile(src, dst)

In [ ]:
print('Train cats images: ', len(os.listdir(train_cats)))
print('Train dogs images: ', len(os.listdir(train_dogs)))

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size = 128,
    class_mode='binary')   

In [ ]:
for data_batch, labels_batch in train_generator:
    print('Data batch shape:', data_batch.shape)
    print('Labels batch shape:', labels_batch.shape)
    break;

In [ ]:
conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(224, 224, 3))

In [ ]:
conv_base.summary()

In [ ]:
conv_base.trainable = False

xem cách lôi lớp layer

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

vâng ạ, em sẽ rút kinh nghiệm ạ

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
             metrics=['acc'])

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
    monitor='acc',
    patience=1),
    keras.callbacks.ModelCheckpoint(
    filepath='my_model.h5',
    monitor='loss',
    svae_best_only=True)
]

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//128,
    epochs = 10,
    callbacks = callbacks_list)

In [ ]:
acc = history.history['acc']
loss = history.history['loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.title('Training accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
filenames = os.listdir("./test1")
test_df = pd.DataFrame({
    'images': filenames
})
nb_samples = test_df.shape[0]
test_df

In [ ]:
test_generator = datagen.flow_from_dataframe(
    test_df, 
    "./test1", 
    x_col ='images',
    y_col = None,
    class_mode=None,
    batch_size=128,
    target_size=(224, 224),
    shuffle=False)

In [ ]:
predict = model.predict(test_generator, steps=12500/128)
threshold = 0.5
test_df['category'] = np.where(predict > threshold, 1,0)

In [ ]:
submission = test_df
submission['id'] = submission['images'].str.split('.').str[0]
submission['label'] = submission['category']
submission.drop(['images', 'category'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)
submission